# 第28课.自定义符号散点图
> Creating a custom ellipse symbol in scatter plot.

在散点图中自定义椭圆形符号。这是一个看似简单，又不简单的例子，很好。

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib widget
matplotlib.rcParams['font.family'] = ['DengXian', 'sans-serif']

## [3]: 准备数据

In [3]:
rx, ry = 3., 1.
area = rx * ry * np.pi
theta = np.arange(0, 2 * np.pi + 0.01, 0.1)
verts = np.column_stack([rx / area * np.cos(theta), ry / area * np.sin(theta)])

- 椭圆的面积是 πab
- 椭圆上任意一点到F1，F2距离的和为2a，F1，F2之间的距离为2c。而公式中的b²=a²-c²。b是为了书写方便设定的参数。
- verts 是一个椭圆的路径点

In [56]:
x, y, s, c = np.random.rand(4, 30)
s *= 10**2.

In [6]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
ax.scatter(x, y, s, c, marker=verts)

完成，收工。

这样看起来这确实是一个简单的例子，但是与*第34课.打标散点图*对比，就能看出不简单的地方。在第34课中讲过marker参数描述标记样式。标记可以是MarkerStyle类的实例，也可以是特定标记的文本简写。缺省为None，此时使用rcParams["scatter.marker"]值，缺省是'o'。

但在上面的例子中verts既不是文本简写，也不是MarkerStyle类的实例，这是怎么回事呢？

In [9]:
isinstance(verts, matplotlib.markers.MarkerStyle)

False

```python
def scatter(self, x, y, s=None, c=None, marker=None ...):
    # load default marker from rcParams
    if marker is None:
        marker = rcParams['scatter.marker']

    if isinstance(marker, mmarkers.MarkerStyle):
        marker_obj = marker
    else:
        marker_obj = mmarkers.MarkerStyle(marker)
```
在scatter方法的定义中，可以看到verts作为MarkerStyle构造器的参数实例化marker对象。

MarkerStyle是markers模块的类，模块包含处理标记的功能，被plot和scatter方法的标记功能使用。在marker所有可能的定义中，

> A list of (x, y) pairs used for Path vertices. The center of the marker is located at (0, 0) and the size is normalized, such that the created path is encapsulated inside the unit cell.

存在一个列表，元素为(x, y)对，被用于路径Path顶点（路点）。标记的中心定位在(0, 0)，大小是标准化后的尺寸，以便将创建的路径封装在单位单元内。

### 实验1 观察verts变量及其对应的marker_obj对象

In [22]:
verts.shape

(63, 2)

In [27]:
verts[0:5,:]

array([[0.31830989, 0.        ],
       [0.31671966, 0.01059265],
       [0.31196488, 0.02107947],
       [0.30409305, 0.03135567],
       [0.29318282, 0.04131857]])

In [9]:
marker_obj = matplotlib.markers.MarkerStyle(verts)

In [34]:
path = marker_obj.get_path()

path没有路径指令？

默认的路径指令可能是，MOVETO,LINETO...,CLOSEPOLY

### 实验2 绘制marker_obj对象

In [15]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
patch = matplotlib.patches.PathPatch(path, facecolor='red', lw=2)

In [36]:
ax.add_patch(patch)

In [20]:
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)

(-1, 1)

### 实验3 
path = marker_obj.get_path().transformed(
            marker_obj.get_transform())

In [31]:
ax.cla()

In [33]:
path = marker_obj.get_path().transformed(
            marker_obj.get_transform())
patch = matplotlib.patches.PathPatch(path, facecolor='orange', lw=2)
ax.add_patch(patch)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)

(-1, 1)

#### transform
**Ref**
- [二维仿射几何变换](https://ww2.mathworks.cn/help/images/ref/affine2d.html)
- [Affine transform of an image](https://matplotlib.org/3.1.3/gallery/images_contours_and_fields/affine_image.html)
- [何为仿射变换(Affine Transformation)](https://www.cnblogs.com/bnuvincent/p/6691189.html)
- [椭圆参数方程](https://jingyan.baidu.com/article/22a299b5c193a99e19376abe.html)
- [仿射变换及其变换矩阵的理解](https://www.cnblogs.com/shine-lee/p/10950963.html)
- 线性变换
- [matplotlib.transforms](https://matplotlib.org/api/transformations.html)
- [Transformations Tutorial](https://matplotlib.org/tutorials/advanced/transforms_tutorial.html#sphx-glr-tutorials-advanced-transforms-tutorial-py)
> from .transforms import IdentityTransform, Affine2D

    def get_transform(self):
        return self._transform.frozen()
        

In [42]:
t = marker_obj.get_transform()

In [43]:
t?

Type:           Affine2D
String form:   
Affine2D(
    [[1.57079633 0.         0.        ]
     [0.         1.57079633 0.        ]
     [0.         0.         1.        ]])
File:           d:\apps\python37\lib\site-packages\matplotlib\transforms.py
Docstring:      A mutable 2D affine transformation.
Init docstring:
Initialize an Affine transform from a 3x3 numpy float array::

  a c e
  b d f
  0 0 1

If *matrix* is None, initialize with the identity transform.


## path
transformed?
```python
    def transformed(self, transform):
        """
        Return a transformed copy of the path.

        See Also
        --------
        matplotlib.transforms.TransformedPath
            A specialized path class that will cache the transformed result and
            automatically update when the transform changes.
        """
        return Path(transform.transform(self.vertices), self.codes,
                    self._interpolation_steps)
```

In [49]:
path._interpolation_steps??

Type:        int
String form: 1
Docstring:  
int([x]) -> integer
int(x, base=10) -> integer

Convert a number or string to an integer, or return 0 if no arguments
are given.  If x is a number, return x.__int__().  For floating point
numbers, this truncates towards zero.

If x is not a number or if base is given, then x must be a string,
bytes, or bytearray instance representing an integer literal in the
given base.  The literal can be preceded by '+' or '-' and be surrounded
by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
Base 0 means to interpret the base from the string as an integer literal.
>>> int('0b100', base=0)
4


In [37]:
marker_obj.get_transform()

In [52]:
t.transform(path.vertices)

array([[ 0.5       ,  0.        ],
       [ 0.49750208,  0.0166389 ],
       [ 0.49003329,  0.03311156],
       [ 0.47766824,  0.04925337],
       [ 0.4605305 ,  0.06490306],
       [ 0.43879128,  0.07990426],
       [ 0.41266781,  0.09410708],
       [ 0.38242109,  0.10736961],
       [ 0.34835335,  0.11955935],
       [ 0.31080498,  0.13055448],
       [ 0.27015115,  0.14024516],
       [ 0.22679806,  0.14853456],
       [ 0.18117888,  0.15533985],
       [ 0.13374941,  0.16059303],
       [ 0.08498357,  0.16424162],
       [ 0.0353686 ,  0.16624916],
       [-0.01459976,  0.1665956 ],
       [-0.06442225,  0.16527747],
       [-0.11360105,  0.16230794],
       [-0.16164478,  0.15771668],
       [-0.20807342,  0.15154957],
       [-0.25242305,  0.14386823],
       [-0.29425056,  0.1347494 ],
       [-0.33313801,  0.1242842 ],
       [-0.36869686,  0.1125772 ],
       [-0.40057181,  0.09974536],
       [-0.42844438,  0.0859169 ],
       [-0.45203607,  0.07122998],
       [-0.47111117,

In [53]:
# 放大的仿射变换
t.get_matrix()

array([[1.57079633, 0.        , 0.        ],
       [0.        , 1.57079633, 0.        ],
       [0.        , 0.        , 1.        ]])

In [54]:
t.identity().get_matrix()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
marker_obj.get_transform()

这是怎么回事？
```python
        # load default marker from rcParams
        if marker is None:
            marker = rcParams['scatter.marker']

        if isinstance(marker, mmarkers.MarkerStyle):
            marker_obj = marker
        else:
            marker_obj = mmarkers.MarkerStyle(marker)

        path = marker_obj.get_path().transformed(
            marker_obj.get_transform())
        if not marker_obj.is_filled():
            edgecolors = 'face'
            linewidths = rcParams['lines.linewidth']

        offsets = np.ma.column_stack([x, y])

        collection = mcoll.PathCollection(
                (path,), scales,
                facecolors=colors,
                edgecolors=edgecolors,
                linewidths=linewidths,
                offsets=offsets,
                transOffset=kwargs.pop('transform', self.transData),
                alpha=alpha
                )
        collection.set_transform(mtransforms.IdentityTransform())
        collection.update(kwargs)

        if colors is None:
            collection.set_array(c)
            collection.set_cmap(cmap)
            collection.set_norm(norm)

            if vmin is not None or vmax is not None:
                collection.set_clim(vmin, vmax)
            else:
                collection.autoscale_None()
                
        self.add_collection(collection)
```
> marker_obj = mmarkers.MarkerStyle(marker)
verts作为MarkerStyle构造器的参数，实例化marker对象

path没有路径指令？

默认的路径指令可能是，MOVETO,LINETO...,CLOSEPOLY

array([0.11301703, 0.05978654, 0.88622347, 0.12624546, 0.7588806 ,
       0.62983259, 0.93101403, 0.45262402, 0.60470987, 0.67459773,
       0.07023867, 0.64647912, 0.43189744, 0.75579181, 0.22463674,
       0.57954772, 0.88872214, 0.13519554, 0.30330362, 0.54826014,
       0.05752122, 0.21296075, 0.20804315, 0.24619614, 0.96239112,
       0.52824613, 0.51499091, 0.88999741, 0.74716556, 0.3298846 ])

In [59]:
np.ma.column_stack([x, y])

masked_array(
  data=[[0.40026150325976384, 0.11301703354550818],
        [0.875427499977598, 0.05978653819814328],
        [0.5672757018129796, 0.8862234672576381],
        [0.9617375668385404, 0.12624545959047395],
        [0.1356499366951729, 0.7588805994008981],
        [0.187725168907667, 0.6298325856327857],
        [0.5070702179092512, 0.9310140304475991],
        [0.07276406819150405, 0.4526240163185016],
        [0.8363450313833953, 0.6047098706459085],
        [0.7459757959954108, 0.6745977329454865],
        [0.6370395744354557, 0.07023867135402095],
        [0.2202030138637866, 0.6464791214399058],
        [0.7780179132077204, 0.43189743791815294],
        [0.6016202419428829, 0.75579181273855],
        [0.017793875414288896, 0.22463673999995482],
        [0.9920190064196774, 0.579547721801578],
        [0.2986503197036011, 0.8887221437123501],
        [0.6059938847592574, 0.13519553529144512],
        [0.23737725362127393, 0.3033036198556047],
        [0.7310325423425654, 

In [61]:
matplotlib.transforms.IdentityTransform().get_matrix()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

collection = mcoll.PathCollection(
                (path,), scales,
                facecolors=colors,
                edgecolors=edgecolors,
                linewidths=linewidths,
                offsets=offsets,
                transOffset=kwargs.pop('transform', self.transData),
                alpha=alpha
                )

### matplotlib.collections

> Classes for the efficient drawing of large collections of objects that share most properties, e.g., a large number of line segments or polygons.

> The classes are not meant to be as flexible as their single element counterparts (e.g., you may not be able to select all line styles) but they are meant to be fast for common use cases (e.g., a large set of solid line segments).

模块中包含的类用于高效绘制大量共享属性的对象，如大量的线段或多边形。

这些类并不像它们的单元素对应物那样灵活（例如，您可能无法选择所有线型），但是对于常见用例（例如，大量的实线线段）而言，它们的作用就是快。

#### class matplotlib.collections.PathCollection(paths, sizes=None, **kwargs)

> This is the most basic Collection subclass. A PathCollection is e.g. created by a scatter() plot.

这是最基本的Collection子类。PathCollection由一下制图方法创建。例如 由scatter（）

> paths is a sequence of matplotlib.path.Path instances.

参数paths是Path实例的序列。

    Valid Collection keyword arguments:

            edgecolors: None
            facecolors: None
            linewidths: None
            antialiaseds: None
            offsets: None
            transOffset: transforms.IdentityTransform()
            norm: None (optional for matplotlib.cm.ScalarMappable)
            cmap: None (optional for matplotlib.cm.ScalarMappable)

    offsets and transOffset are used to translate the patch after rendering (default no offsets)

    If any of edgecolors, facecolors, linewidths, antialiaseds are None, they default to their matplotlib.rcParams patch setting, in sequence form.


In [68]:
__name__

'__main__'